In [1]:
!pip install beautifulsoup4
!pip install requests
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
#Get the html data from Wiki 
url=requests.get('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050.').text

In [3]:
soup=BeautifulSoup(url,'html')

In [4]:
list=[]

In [5]:
#find in the html the table and clean the data, save the data in a list
trs = soup.find_all('tr')
for i in trs:
    i.find_all('td')
    text = str(i)
    text = text.replace("<td style","")
    text = text.replace("width:11%; vertical-align:top; color:#ccc;","")
    text = text.replace("<p><b>","")
    text = text.replace("</b><br/><span style=","")
    text = text.replace("font-size:80%;","")  
    text = text.replace("</p>","")
    text = text.replace("</td>","") 
    text = text.replace("</span>","")  
    text = text.replace("width:11%; vertical-align:top;","")
    text = text.replace("=","")
    text = text.replace("<a ","")
    text = text.replace("</a","")
    text = text.replace("<tr","")
    text = text.replace("</th","")
    text = text.replace("<td","")
    text = text.replace("\n","")
    text = text.replace("<i","")
    text = text.replace("vertical-align:top;","")
    text = text.split(">")
    list.append(text)
    #print(text)

In [6]:
#Transform the list into a DataFrame
df=pd.DataFrame(list)

In [7]:
#Clean the usseles columns and use the first row as headers
col_list = [2, 4,5,7, 6]
df=df[col_list]
new_header = df.iloc[0] #grab the first row for the header
df = df[1:] #take the data less the header row
df.columns = new_header #set the header row as the df header
df = df[~(df['Borough'] == 'Not assigned</tr')]


In [8]:
import numpy as np

In [9]:
df["Neighborhood"]=np.where(df['</tr']=='', df['Neighbourhood'],df['</tr'])
col_list = ['Postcode', 'Borough', "Neighborhood"]
df=df[col_list]

In [10]:
#make a groupby in order to separate the neighborhoods by coma in an unique field
df['Neighborhood'] = df['Neighborhood'].astype(str)
df=df.groupby(['Postcode','Borough'])['Neighborhood'].apply(','.join).reset_index()

In [11]:
#Filtering by the Postocode fields
df = df[df['Postcode'].str.contains('M', na=False)]

In [12]:
df

,Postcode,Borough,Neighborhood
1,M1B,Scarborough,"Rouge,Malvern"
2,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
3,M1E,Scarborough,"Guildwood</tr,Morningside,West Hill"
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae</tr
...,...,...,...
99,M9N,York,Weston
100,M9P,Etobicoke,Westmount</tr
101,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens</tr,Ric..."
102,M9V,Etobicoke,"Albion Gardens</tr,Beaumond Heights,Humbergate..."
